## Alunos: Ícaro Pires (15/0129815) e Vinicius Lima (15/0151331)

# Lista 01 - Algoritmos de Busca - EDAII

## Meet in the middle

### Exemplo de aplicação prática: forças atuantes sobre um objeto

Imagine que em um dado objeto haja um conjunto de forças atuando sobre este. Deseja-se descobrir quais forças (vetores) que quando somadas, geram uma força resultante máxima mas sem comprometer a estrutura do objeto.

Supondo que seja conhecida a força máxima = MAXIMUM_VALUE e o conjunto de forças atuantes sobre o objeto = S, o problema poderia ser resolvido gerando as somas de todos os subconjuntos de S e selecionando o subconjunto que mais se aproxima de MAXIMUM_VALUE.

Utilizando uma estratégia naive o resultado poderia ser encontrado para conjuntos pequenos de forças em tempo viável devido sua alta complexidade de tempo O(2^(n)).

Para viabilizar o cálculo para conjuntos de forças um pouco maiores, podemos usar a técnica meet in the middle que utiliza da busca binária para reduzir em parte a complexidade do problema.

A técnica consiste basicamente em:
* Dividir o conjunto em 2 subconjuntos cada um com a metade dos elementos;
* Gerar a soma dos possíveis subconjuntos dos dois subconjuntos gerados anteriormente e armazená-los em vetores;
* Em seguida, ordena-se um dos vetores para viabilizar a busca binária;
* Para cada elemento do vetor não ordenado, buscar o elemento do vetor ordenado que melhor atende o problema.

Para a busca binária, utiliza-se especificamente o algoritimo de busca lower bound. O motivo para isso é o de que caso não se encontre exatamente o valor desejado, possa-se encontrar o mais próximo possível que não o exceda.

### Implementação

In [1]:
import numpy as np

# Constantes
MAXIMUM_VALUE = 32156  # Valor limite para a soma das forças sem compromoter o objeto
SET_LOWER_BOUND, SET_UPPER_BOUND = int(-1e8), int(1e8)  # Números máximos e mínimos do conjunto de teste
SET_SIZE = 36  # tamanho do conjunto de testes

# Gerando conjunto base
S = np.random.randint(low=SET_LOWER_BOUND, high=SET_UPPER_BOUND, size=SET_SIZE)

In [2]:
# Separando subconjuntos
mid = SET_SIZE//2
subset_a, subset_b = S[:mid], S[mid:]

In [3]:
from collections import namedtuple

def generate_all_subsets_sums(s, *, is_first_half):
    VectorPosition = namedtuple('VectorPosition', ['value', 'index'])
    sub_set_size, subsets_sums = len(s), []

    for i in range(1<<sub_set_size):
        ssum, sub_set = 0, set()
        for j in range(sub_set_size):
            if i & (1<<j):
                ssum += s[j]
                if is_first_half:
                    sub_set.add(VectorPosition(s[j], j))
                else:
                    sub_set.add(VectorPosition(s[j], SET_SIZE//2 + j))
        subsets_sums += [(ssum, sub_set)]
    return subsets_sums

In [4]:
# Gerando possíveis somas de cada subconjunto
subset_a_possible_sums = generate_all_subsets_sums(subset_a, is_first_half=True)
subset_b_possible_sums = generate_all_subsets_sums(subset_b, is_first_half=False)

In [5]:
def lower_bound(xs, element):
    if element > xs[-1][0]:
        return -1

    def lower_bound_internal(l, r):  # Usa busca binária
        mid = l + (r-l)//2
        
        if element == xs[mid][0] or l == r:
            return mid
        
        if element > xs[mid][0]:
            return lower_bound_internal(mid+1, r)
        return lower_bound_internal(l, mid)

    return lower_bound_internal(0, len(xs)-1)

In [6]:
def find_value(subset_a, subset_b):
    mmax = (SET_LOWER_BOUND - 1, set())
    subset_a_possible_sums.sort()

    for element, sset in subset_b:
        if element == MAXIMUM_VALUE:
            return (element, sset)
        
        if element < MAXIMUM_VALUE:
            looked = MAXIMUM_VALUE - element
            looked_idx = lower_bound(subset_a, looked)

            if looked_idx != -1 and subset_a[looked_idx][0] <= looked:
                subset_value, subset_set = subset_a[looked_idx]
                mmax = max(mmax, (subset_value + element, subset_set))

    return mmax

In [7]:
# Calculando resultado
result_value, result_set = find_value(subset_a_possible_sums, subset_b_possible_sums)

# Imprimindo resultados
if result_value != SET_LOWER_BOUND - 1:
    print(f'Valor mais próximo possível: {result_value}', end='\n')
    
    print('Vetor original: [\n\t{}\n]\n'.format(',\n\t'.join(
        map(str,list(enumerate(S)))))
    )
    
    print('Elementos eleitos:\n {}'.format('\n'.join(
        ['\tValue = {:10d}, Index = {:2d}'.format(int(value), int(index)) 
         for value, index in result_set]
    )))
else:
    print('Não foi encontrado nenhum valor próximo ao desejado e menor.')

Valor mais próximo possível: 32156
Vetor original: [
	(0, 26075349),
	(1, -51069202),
	(2, -93477838),
	(3, -2557455),
	(4, -18523952),
	(5, -62344640),
	(6, -16643973),
	(7, 5243471),
	(8, -38799230),
	(9, -8286166),
	(10, -51707725),
	(11, -26585093),
	(12, -47198476),
	(13, 62305900),
	(14, 90324998),
	(15, -78840746),
	(16, 70747759),
	(17, -19424769),
	(18, -92140340),
	(19, -77131025),
	(20, 34815810),
	(21, -11354310),
	(22, -68540328),
	(23, 87545538),
	(24, -71667735),
	(25, 48306360),
	(26, 75248885),
	(27, 15213568),
	(28, -64142569),
	(29, 48766918),
	(30, 79202780),
	(31, -37683520),
	(32, 29738368),
	(33, 54079580),
	(34, 28877669),
	(35, 44679858)
]

Elementos eleitos:
 	Value =   26075349, Index =  0
	Value =  -18523952, Index =  4
	Value =   62305900, Index = 13
	Value =  -38799230, Index =  8
	Value =  -78840746, Index = 15
	Value =  -16643973, Index =  6
	Value =   -2557455, Index =  3
	Value =  -26585093, Index = 11
	Value =  -51707725, Index = 10
	Value =   7074775

### Conclusões

Com base no código acima, pode-se observar a técnica sendo aplicada num conjunto de 36 posições. Caso a técnica naive tivesse sido utilizada, seria necessário gerar 2^36 = 68.719.476.736 subconjuntos, enquanto que geramos apenas 2^18 * 2 = 524.288 subconjuntos, uma diferença notável. Além de ganhar desempenho através da busca binária.

### Referências

Meet in the middle: https://www.geeksforgeeks.org/meet-in-the-middle/